In [7]:
import pandas as pd
import json

In [8]:
with open("./output/products.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [9]:
def extract_description(desc):
    try:
        desc_json = json.loads(desc)  # parse stringified JSON
        blocks = desc_json.get("blocks", [])
        texts = [block["data"].get("text", "") for block in blocks]
        return " ".join(texts).replace("\u2022", "•")  # keep bullet points clean
    except Exception:
        return desc

In [10]:
rows = []
for product in data:
    base_info = {
        "product_id": product["id"],
        "product_name": product["name"],
        "category_id": product["category_id"],
        "product_type_id": product["product_type_id"],
        "description": extract_description(product["description"]),
    }
    
    for variant in product.get("variants", []):
        for listing in variant.get("channelListings", []):
            row = base_info.copy()
            row.update({
                "variant_id": variant["variant_id"],
                "variant_name": variant["variant_name"],
                "channel_id": listing["channel_id"],
                "channel_name": listing["channel_name"],
                "currency": listing["currency"],
                "price": listing["price"],
            })
            rows.append(row)

In [11]:
df = pd.DataFrame(rows)

In [12]:
output_file = "./output/list_of_products.xlsx"
df.to_excel(output_file, index=False)

print(f"✅ Excel file saved as {output_file}")

✅ Excel file saved as ./output/list_of_products.xlsx
